In [132]:
# Install stopwords package
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.stem import WordNetLemmatizer
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Sidharth
[nltk_data]     Veluvolu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sidharth
[nltk_data]     Veluvolu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sidharth
[nltk_data]     Veluvolu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [80]:
df = pd.ExcelFile('training-Obama-Romney-tweets.xlsx')

In [81]:
df

In [82]:
df1 = pd.read_excel(df, 'Obama')
df2 = pd.read_excel(df, 'Romney')

In [83]:
df1

,Unnamed: 0,date,time,Anootated tweet,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,"1: positive, -1: negative, 0: neutral, 2: mixed",Class,Your class
1,NaN,2012-10-16 00:00:00,10:28:53-05:00,"Kirkpatrick, who wore a baseball cap embroider...",0,NaN
2,NaN,2016-12-10 00:00:00,10:09:00-05:00,Question: If <e>Romney</e> and <e>Obama</e> ha...,2,NaN
3,NaN,2012-10-16 00:00:00,10:04:30-05:00,#<e>obama</e> debates that Cracker Ass Cracker...,1,NaN
4,NaN,2012-10-16 00:00:00,10:00:36-05:00,RT @davewiner Slate: Blame <e>Obama</e> for fo...,2,NaN
...,...,...,...,...,...,...
7194,NaN,10/17/2012,AM 11:7:09,The Reason <e>Ann Romney</e> And <e>Michelle ...,0,NaN
7195,NaN,10/17/2012,AM 11:9:13,<e>Obama</e> Kenakan Cincin Syahadat Sejak SM...,0,NaN
7196,NaN,10/17/2012,AM 11:11:34,"Bitches be like ""Obama<3"" bitches just want <...",0,NaN
7197,NaN,10/17/2012,AM 11:13:16,<e>president</e> Barack <e>Obama</e> and Repu...,2,NaN


In [84]:
df2

,Unnamed: 0,date,time,Anootated tweet,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,"1: positive, -1: negative, 0: neutral, 2: mixed",Class,Your class label
1,NaN,2012-10-16 00:00:00,09:38:08-05:00,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1,NaN
2,NaN,2012-10-16 00:00:00,10:22:34-05:00,Senior <e>Romney</e> Advisor Claims <e>Obama</...,2,NaN
3,NaN,2012-10-16 00:00:00,10:14:18-05:00,.@WardBrenda @shortwave8669 @allanbourdius you...,-1,NaN
4,NaN,2012-10-16 00:00:00,09:27:16-05:00,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1,NaN
...,...,...,...,...,...,...
7196,NaN,2012-10-17 00:00:00,AM 11:14:20,"CNN's John King: After Two Debates"""" <e>Romney...",1,NaN
7197,NaN,2012-10-17 00:00:00,AM 11:15:53,el 59 por ciento de las mujeres blancas casada...,0,NaN
7198,NaN,2012-10-17 00:00:00,AM 11:18:27,@FedericoArreola No vimos perder a <e>Romney</...,2,NaN
7199,NaN,2012-10-17 00:00:00,AM 11:20:04,"""And they brought us a whole binder of women""""...",-1,NaN


In [85]:
df1 = df1.drop(df1.columns[[0,1,2]], axis = 1)#before='Anootated tweet', after = 'Unnamed: 5', axis = 1)

In [86]:
df2 = df2.drop(df2.columns[[0,1,2]], axis = 1)#before='Anootated tweet', after = 'Unnamed: 5', axis = 1)

In [87]:
# # Concatenating the two sheets in excel
# initial = [df1,df2]
# for i in initial:
#     i.columns = ['Anootated tweet', 'Given class', 'Result']
# initial = pd.concat(initial).reset_index(drop = True)

In [88]:
# # Concatenating the two sheets in excel
for i in [df1]:
    i.columns = ['Anootated tweet', 'Given class', 'Result']

In [89]:
# # Concatenating the two sheets in excel
for i in [df2]:
    i.columns = ['Anootated tweet', 'Given class', 'Result']

In [90]:
df1

,Anootated tweet,Given class,Result
0,"1: positive, -1: negative, 0: neutral, 2: mixed",Class,Your class
1,"Kirkpatrick, who wore a baseball cap embroider...",0,NaN
2,Question: If <e>Romney</e> and <e>Obama</e> ha...,2,NaN
3,#<e>obama</e> debates that Cracker Ass Cracker...,1,NaN
4,RT @davewiner Slate: Blame <e>Obama</e> for fo...,2,NaN
...,...,...,...
7194,The Reason <e>Ann Romney</e> And <e>Michelle ...,0,NaN
7195,<e>Obama</e> Kenakan Cincin Syahadat Sejak SM...,0,NaN
7196,"Bitches be like ""Obama<3"" bitches just want <...",0,NaN
7197,<e>president</e> Barack <e>Obama</e> and Repu...,2,NaN


In [91]:
df2

,Anootated tweet,Given class,Result
0,"1: positive, -1: negative, 0: neutral, 2: mixed",Class,Your class label
1,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1,NaN
2,Senior <e>Romney</e> Advisor Claims <e>Obama</...,2,NaN
3,.@WardBrenda @shortwave8669 @allanbourdius you...,-1,NaN
4,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1,NaN
...,...,...,...
7196,"CNN's John King: After Two Debates"""" <e>Romney...",1,NaN
7197,el 59 por ciento de las mujeres blancas casada...,0,NaN
7198,@FedericoArreola No vimos perder a <e>Romney</...,2,NaN
7199,"""And they brought us a whole binder of women""""...",-1,NaN


In [92]:
#Converting text to lower case
df1['Anootated tweet'] = df1['Anootated tweet'].str.lower()

In [93]:
#Converting text to lower case
df2['Anootated tweet'] = df2['Anootated tweet'].str.lower()

In [94]:
# list of stop words
stop_list = stopwords.words('english')

In [95]:
# Remove stop words from text
df1['Anootated tweet'] = df1['Anootated tweet'].apply(lambda x: " ".join([word for word in str(x).split() if word not in stop_list]))

In [96]:
# Remove stop words from text
df2['Anootated tweet'] = df2['Anootated tweet'].apply(lambda x: " ".join([word for word in str(x).split() if word not in stop_list]))

In [97]:
tweets = []
for tweet in df1['Anootated tweet']:
    tweet = re.sub("@[A-Za-z0-9_*]+","", str(tweet))
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    tweet = tweet.replace("#", "").replace("_", " ").replace("'", "")
    tweet = re.sub("[\W_]", " ", tweet)    
    tweet = word_tokenize(tweet)
    tweet = " ".join(tweet)
    tweets.append(tweet)
df1['Anootated tweet'] = tweets

In [98]:
tweets = []
for tweet in df2['Anootated tweet']:
    tweet = re.sub("@[A-Za-z0-9_*]+","", str(tweet))
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    tweet = tweet.replace("#", "").replace("_", " ").replace("'", "")
    tweet = re.sub("[\W_]", " ", tweet)    
    tweet = word_tokenize(tweet)
    tweet = " ".join(tweet)
    tweets.append(tweet)
df2['Anootated tweet'] = tweets

In [99]:
#Stemming
st = PorterStemmer()
df1['Anootated tweet'] = df1['Anootated tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


In [100]:
#Stemming
df2['Anootated tweet'] = df2['Anootated tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


In [101]:
#Deleting single length characters
df1['Anootated tweet'] = df1['Anootated tweet'].apply(lambda x: " ".join([word for word in x.split() if len(word)>1]))

In [102]:
#Deleting single length characters
df2['Anootated tweet'] = df2['Anootated tweet'].apply(lambda x: " ".join([word for word in x.split() if len(word)>1]))

In [103]:
df3 = df1[df1['Given class'].isin([-1, 0, 1])]

In [104]:
df4 = df2[df2['Given class'].isin([-1, 0, 1])]

In [105]:
#df1
df3.head()

,Anootated tweet,Given class,Result
1,kirkpatrick wore basebal cap embroid barack ob...,0,NaN
3,obama debat cracker ass cracker tonight tune i...,1,NaN
5,your miss point im afraid understand bigger pi...,0,NaN
7,rais democrat left parti year ago 1980 lifetim...,-1,NaN
8,obama camp cant afford lower expect tonight de...,0,NaN


In [106]:
#df2
df4.head()

,Anootated tweet,Given class,Result
1,insidi mitt romney bain help philip morri get ...,-1,NaN
3,mean like romney cheat primari,-1,NaN
4,mitt romney still believ have black presid,-1,NaN
5,romney tax plan deserv 2nd look secret one tha...,-1,NaN
6,hope romney debat prep peopl last time,1,NaN


In [107]:
print(df3.shape)
print(df4.shape)

(5471, 3)
(5648, 3)


In [108]:
df3.dtypes

Anootated tweet    object
Given class        object
Result             object
dtype: object

In [109]:
df4.dtypes

Anootated tweet    object
Given class        object
Result             object
dtype: object

In [110]:
df3 = df3.infer_objects()

In [111]:
df4 = df4.infer_objects()

In [112]:
df3.dtypes

Anootated tweet     object
Given class          int64
Result             float64
dtype: object

In [113]:
df4.dtypes

Anootated tweet     object
Given class          int64
Result             float64
dtype: object

In [114]:
def get_feature_vector(train_fit):
     vector = TfidfVectorizer(sublinear_tf=True)
     vector.fit(train_fit)
     return vector

In [115]:
tf_vector1 = get_feature_vector(np.array(df3.iloc[:, 0]).ravel())
X1 = tf_vector1.transform(np.array(df3.iloc[:, 0]).ravel())
Y1 = np.array(df3.iloc[:, 1]).ravel()

In [116]:
tf_vector2 = get_feature_vector(np.array(df4.iloc[:, 0]).ravel())
X2 = tf_vector2.transform(np.array(df4.iloc[:, 0]).ravel())
Y2 = np.array(df4.iloc[:, 1]).ravel()

In [117]:
X1

<5471x6924 sparse matrix of type '<class 'numpy.float64'>'
	with 49273 stored elements in Compressed Sparse Row format>

In [118]:
Y1

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [119]:
X2

<5648x6438 sparse matrix of type '<class 'numpy.float64'>'
	with 51794 stored elements in Compressed Sparse Row format>

In [120]:
Y2

array([-1, -1, -1, ...,  0, -1,  1], dtype=int64)

In [121]:
# # Dividing it into training and validation and then to implement the model - Obama
X_train1, X_valid1, Y_train1, Y_valid1 = train_test_split(X1, Y1, test_size = 0.20, random_state = 0)

In [122]:
# # Dividing it into training and validation and then to implement the model - Romney
X_train2, X_valid2, Y_train2, Y_valid2 = train_test_split(X2, Y2, test_size = 0.20, random_state = 0)

In [123]:
# Bayesian model
NB_model = MultinomialNB()
NB_model.fit(X_train1, Y_train1)
y_predict_nb = NB_model.predict(X_valid1)
print(accuracy_score(Y_valid1, y_predict_nb))

0.589041095890411


In [124]:
# Bayesian model
NB_model = MultinomialNB()
NB_model.fit(X_train2, Y_train2)
y_predict_nb = NB_model.predict(X_valid2)
print(accuracy_score(Y_valid2, y_predict_nb))

0.5601769911504425


In [125]:
# Linear regression model
LR_model = LogisticRegression(solver='lbfgs',max_iter=10000)
LR_model.fit(X_train1, Y_train1)
y_predict_lr = LR_model.predict(X_valid1)
print(accuracy_score(Y_valid1, y_predict_lr))
print(confusion_matrix(Y_valid1, y_predict_lr))
print(classification_report(Y_valid1, y_predict_lr))

0.6036529680365297
[[255  99  37]
 [ 88 209  66]
 [ 51  93 197]]
              precision    recall  f1-score   support

          -1       0.65      0.65      0.65       391
           0       0.52      0.58      0.55       363
           1       0.66      0.58      0.61       341

    accuracy                           0.60      1095
   macro avg       0.61      0.60      0.60      1095
weighted avg       0.61      0.60      0.60      1095



In [126]:
# Linear regression model
LR_model = LogisticRegression(solver='lbfgs',max_iter=10000)
LR_model.fit(X_train2, Y_train2)
y_predict_lr = LR_model.predict(X_valid2)
print(accuracy_score(Y_valid2, y_predict_lr))
print(confusion_matrix(Y_valid2, y_predict_lr))
print(classification_report(Y_valid2, y_predict_lr))

0.5902654867256637
[[497  64  17]
 [198 104  24]
 [117  43  66]]
              precision    recall  f1-score   support

          -1       0.61      0.86      0.72       578
           0       0.49      0.32      0.39       326
           1       0.62      0.29      0.40       226

    accuracy                           0.59      1130
   macro avg       0.57      0.49      0.50      1130
weighted avg       0.58      0.59      0.56      1130



In [130]:
# Random forest model - Obama
RF_model = RandomForestClassifier(n_estimators=200)
RF_model.fit(X_train1,Y_train1)
y_predict_rf = RF_model.predict(X_valid1)
print(accuracy_score(Y_valid1, y_predict_rf))
print(confusion_matrix(Y_valid1, y_predict_rf))
print(classification_report(Y_valid1, y_predict_rf))


0.5744292237442923
[[204 136  51]
 [ 64 242  57]
 [ 48 110 183]]
              precision    recall  f1-score   support

          -1       0.65      0.52      0.58       391
           0       0.50      0.67      0.57       363
           1       0.63      0.54      0.58       341

    accuracy                           0.57      1095
   macro avg       0.59      0.58      0.57      1095
weighted avg       0.59      0.57      0.57      1095



In [131]:
# Random forest model - Romney
RF_model = RandomForestClassifier(n_estimators=200)
RF_model.fit(X_train2,Y_train2)
y_predict_rf = RF_model.predict(X_valid2)
print(accuracy_score(Y_valid2, y_predict_rf))
print(confusion_matrix(Y_valid2, y_predict_rf))
print(classification_report(Y_valid2, y_predict_rf))

0.5814159292035398
[[514  54  10]
 [233  74  19]
 [135  22  69]]
              precision    recall  f1-score   support

          -1       0.58      0.89      0.70       578
           0       0.49      0.23      0.31       326
           1       0.70      0.31      0.43       226

    accuracy                           0.58      1130
   macro avg       0.59      0.47      0.48      1130
weighted avg       0.58      0.58      0.54      1130



In [135]:
# support vector classification - Obama
SVCmodel = SVC()
SVCmodel.fit(X_train1,Y_train1)
y_predict_svc = SVCmodel.predict(X_valid1)
print(accuracy_score(Y_valid1, y_predict_svc))
print(confusion_matrix(Y_valid1, y_predict_svc))
print(classification_report(Y_valid1, y_predict_svc))

0.6136986301369863
[[264  99  28]
 [ 86 229  48]
 [ 54 108 179]]
              precision    recall  f1-score   support

          -1       0.65      0.68      0.66       391
           0       0.53      0.63      0.57       363
           1       0.70      0.52      0.60       341

    accuracy                           0.61      1095
   macro avg       0.63      0.61      0.61      1095
weighted avg       0.63      0.61      0.61      1095



In [136]:
# support vector classification - Romney
SVCmodel = SVC()
SVCmodel.fit(X_train2,Y_train2)
y_predict_svc = SVCmodel.predict(X_valid2)
print(accuracy_score(Y_valid2, y_predict_svc))
print(confusion_matrix(Y_valid2, y_predict_svc))
print(classification_report(Y_valid2, y_predict_svc))

0.588495575221239
[[541  30   7]
 [238  68  20]
 [154  16  56]]
              precision    recall  f1-score   support

          -1       0.58      0.94      0.72       578
           0       0.60      0.21      0.31       326
           1       0.67      0.25      0.36       226

    accuracy                           0.59      1130
   macro avg       0.62      0.46      0.46      1130
weighted avg       0.60      0.59      0.53      1130



In [75]:
# Dividing it into training and validation and then to implement the model - Obama
X_train1, X_valid1, Y_train1, Y_valid1 = train_test_split(df3['Anootated tweet'].values, df3['Given class'].values, test_size = 0.20, random_state = 0)

In [76]:
# Dividing it into training and validation and then to implement the model - Romney
X_train2, X_valid2, Y_train2, Y_valid2 = train_test_split(df4['Anootated tweet'].values, df4['Given class'].values, test_size = 0.20, random_state = 0)

In [77]:
# Bayesian model - Obama
nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train1, Y_train1)
y_pred_nb = nb.predict(X_valid1)
print(accuracy_score(Y_valid1, y_pred_nb))
print(confusion_matrix(Y_valid1, y_pred_nb))
print(classification_report(Y_valid1, y_pred_nb))

0.6009132420091324
[[286  72  33]
 [112 185  66]
 [ 68  86 187]]
              precision    recall  f1-score   support

          -1       0.61      0.73      0.67       391
           0       0.54      0.51      0.52       363
           1       0.65      0.55      0.60       341

    accuracy                           0.60      1095
   macro avg       0.60      0.60      0.60      1095
weighted avg       0.60      0.60      0.60      1095



In [78]:
# Bayesian model - Romney
nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train2, Y_train2)
y_pred_nb = nb.predict(X_valid2)
print(accuracy_score(Y_valid2, y_pred_nb))
print(confusion_matrix(Y_valid2, y_pred_nb))
print(classification_report(Y_valid2, y_pred_nb))

0.5530973451327433
[[569   7   2]
 [294  28   4]
 [191   7  28]]
              precision    recall  f1-score   support

          -1       0.54      0.98      0.70       578
           0       0.67      0.09      0.15       326
           1       0.82      0.12      0.22       226

    accuracy                           0.55      1130
   macro avg       0.68      0.40      0.35      1130
weighted avg       0.63      0.55      0.44      1130



In [37]:
# Logistic regression model - Obama
logreg_Obama = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=1000)),
               ])
logreg_Obama.fit(X_train1, Y_train1)
y_pred_log_Obama = logreg_Obama.predict(X_valid1)
print(accuracy_score(Y_valid1, y_pred_log_Obama))
print(confusion_matrix(Y_valid1, y_pred_log_Obama))
print(classification_report(Y_valid1, y_pred_log_Obama))

0.6191780821917808
[[256  80  55]
 [ 89 205  69]
 [ 44  80 217]]
              precision    recall  f1-score   support

          -1       0.66      0.65      0.66       391
           0       0.56      0.56      0.56       363
           1       0.64      0.64      0.64       341

    accuracy                           0.62      1095
   macro avg       0.62      0.62      0.62      1095
weighted avg       0.62      0.62      0.62      1095



In [38]:
# Logistic regression model - Romney
logreg_Romney = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=1000)),
               ])
logreg_Romney.fit(X_train2, Y_train2)
y_pred_log_Romney = logreg_Romney.predict(X_valid2)
print(accuracy_score(Y_valid2, y_pred_log_Romney))
print(confusion_matrix(Y_valid2, y_pred_log_Romney))
print(classification_report(Y_valid2, y_pred_log_Romney))

0.6070796460176991
[[511  50  17]
 [203 103  20]
 [120  34  72]]
              precision    recall  f1-score   support

          -1       0.61      0.88      0.72       578
           0       0.55      0.32      0.40       326
           1       0.66      0.32      0.43       226

    accuracy                           0.61      1130
   macro avg       0.61      0.51      0.52      1130
weighted avg       0.60      0.61      0.57      1130



In [51]:
# print(y_pred_log)

[-1  0  1 ... -1  0 -1]


In [52]:
# print(X_valid2)

['real question america mind size binder romney prefer'
 'billi graham pray romney victori'
 'ross perot endors mitt romney washington time' ...
 'give romney pinocchio bogu math claim hell produc 12 million job hash meat'
 'barack obama mitt romney sing hot cold kati perri'
 'romney tell control super pac slush fund lie ppl mormon']


Test Case

In [ ]:
# tf = pd.ExcelFile('tomorrowtest.xlsx')

In [99]:
test_path = 'C:\\Users\\Sidharth Veluvolu\\Desktop\\'
tf1 = pd.read_excel(test_path + 'final-testData-no-label-Obama-tweets.xlsx', header = None)
tf2 = pd.read_excel(test_path + 'final-testData-no-label-Romney-tweets.xlsx', header = None)

In [100]:
#Converting text to lower case
tf1['Anootated tweet'] = tf1['Anootated tweet'].str.lower()

In [ ]:
#Converting text to lower case
tf2['Anootated tweet'] = tf2['Anootated tweet'].str.lower()

In [101]:
# Remove stop words from text
tf1['Anootated tweet'] = tf1['Anootated tweet'].apply(lambda x: " ".join([word for word in str(x).split() if word not in stop_list]))

In [ ]:
# Remove stop words from text
tf2['Anootated tweet'] = tf2['Anootated tweet'].apply(lambda x: " ".join([word for word in str(x).split() if word not in stop_list]))

In [102]:
#Obama
tweets = []
for tweet in tf1['Anootated tweet']:
    tweet = re.sub("@[A-Za-z0-9_*]+","", str(tweet))
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    tweet = tweet.replace("#", "").replace("_", " ").replace("'", "")
    tweet = re.sub("[\W_]", " ", tweet)    
    tweet = word_tokenize(tweet)
    tweet = " ".join(tweet)
    tweets.append(tweet)
tf1['Anootated tweet'] = tweets

In [ ]:
#Romney
tweets = []
for tweet in tf2['Anootated tweet']:
    tweet = re.sub("@[A-Za-z0-9_*]+","", str(tweet))
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    tweet = tweet.replace("#", "").replace("_", " ").replace("'", "")
    tweet = re.sub("[\W_]", " ", tweet)    
    tweet = word_tokenize(tweet)
    tweet = " ".join(tweet)
    tweets.append(tweet)
tf2['Anootated tweet'] = tweets

In [103]:
tf1['Anootated tweet'] = tf1['Anootated tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


In [ ]:
tf2['Anootated tweet'] = tf2['Anootated tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


In [104]:
tf1['Anootated tweet'] = tf1['Anootated tweet'].apply(lambda x: " ".join([word for word in x.split() if len(word)>1]))

In [ ]:
#Deleting single length characters
tf2['Anootated tweet'] = tf2['Anootated tweet'].apply(lambda x: " ".join([word for word in x.split() if len(word)>1]))

In [105]:
X_test1 = tf1['Anootated tweet']
X_test2 = tf2['Anootated tweet']

In [109]:
y_pred1 = logreg_Obama.predict(X_test1)

In [ ]:
y_pred2 = logreg_Romney.predict(X_test2)

In [110]:
with open(test_path + 'obama.txt', 'w') as f:
    for idx, y in enumerate(y_pred1):
        f.write(str(idx+1) + ';;' + str(y) + '\n')

In [ ]:
with open(test_path + 'romney.txt', 'w') as f:
    for idx, y in enumerate(y_pred2):
        f.write(str(idx+1) + ';;' + str(y) + '\n')